# Exercise Sheet 5 - Segmentation & Denoising with U-Net

 * Deep Learning
 * Instructor: Constantin Pape
 * Due date: **Mon, July 04, noon**

In this assignment we will use a U-Net for two different tasks:
- semantic and instance segmentation of nuclei in microscopy images
- denoising of natural images

## Hints

The goal of the first part of the exercise (segmentation) is to implement the U-Net architecture very similar to the original publication and then use it for its initial purpose, semantic segmentation. In the second part we will use the same U-Net implementation, but use it for a different task: denoising, following the ideas of noise-to-noise. 

To understand the background of this exercise you can:

* Review the lecture (Lecture 7 for U-Net architecture, Lecture 8 for noise-to-noise)
* Read the [U-net publication](https://arxiv.org/abs/1505.04597).
* Read the [noise-to-noise publication](https://arxiv.org/abs/1803.04189).

Note that we will implement the same ideas as in these papers, but will deviate from the implementation details and conduct different and fewer experiments.
 
*Do not hesitate to ask questions and ideally discuss them with your fellow students and me on Rocket Chat! I will monitor the channel to provide you help if your discussions get stuck.*

### IMPORTANT SUBMISSION INSTRUCTIONS

- When you're done, download the notebook and rename it to \<surname1\>_\<surname2\>_\<surname3\>.ipynb
- Only submit the ipynb file, no other file is required
- Submit only once
- The deadline is strict

Implementation
- Only change code to replace placeholders. Leave the other code as is. In addition to the python packages loaded below you are allowed to use any packages you want.


**Import required libraries.**

In [1]:
%load_ext tensorboard
import os
import zipfile
from glob import glob
from pathlib import Path

import imageio
import matplotlib.pyplot as plt
import numpy as np
import requests
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from matplotlib import colors
from scipy.optimize import linear_sum_assignment
from skimage.measure import label
from skimage.metrics import contingency_table, peak_signal_noise_ratio
from skimage.segmentation import find_boundaries, watershed
from skimage.util import random_noise
from google.colab import drive

In [2]:
# mount google drive and create a root folder where to save the data for this exercise
drive.mount("/content/drive")
root_folder = "/content/drive/MyDrive/dlforcv-ex5"
os.makedirs(root_folder, exist_ok=True)

Mounted at /content/drive


## Segmentation

In the first part of the exercise we will implement the U-Net architecture and apply it to a segmentation problem in microscopy: segmenting nuclei in fluorescence images. The data we are using is a subset of the [Kaggle Nucleus Segmentation challenge](https://www.kaggle.com/c/data-science-bowl-2018).

### Data preparation

In [3]:
# download the data
data_folder = os.path.join(root_folder, "kaggle-data")
url = "https://github.com/stardist/stardist/releases/download/0.1.0/dsb2018.zip"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
    tmp_path = os.path.join(data_folder, "data.zip")
    with requests.get(url) as r:
        with open(tmp_path, "wb") as f:
            f.write(r.content)
    with zipfile.ZipFile(tmp_path, "r") as f:
        f.extractall(data_folder)
    os.remove(tmp_path)
data_folder = os.path.join(data_folder, "dsb2018")
assert os.path.exists(data_folder)

**Data exploration**

The data is stored in the root folder `dsb2018` and contains the two subfolders `train` and `test` with train/test split. Each of these folders contains the folders `images` with the raw image data and `masks` with the instance masks. As a first step, we will visualize some of the images and labels from the training data.

In [4]:
image_paths = glob(os.path.join(data_folder, "train", "images", "*.tif"))
image_paths.sort()
mask_paths = glob(os.path.join(data_folder, "train", "masks", "*.tif"))
mask_paths.sort()
assert len(image_paths) == len(mask_paths)

In [5]:
# a function to generate a random color map for a label image
def get_random_colors(labels):
    n_labels = len(np.unique(labels)) - 1
    cmap = [[0, 0, 0]] + np.random.rand(n_labels, 3).tolist()
    cmap = colors.ListedColormap(cmap)
    return cmap

In [6]:
def plot_sample(image_path, mask_path):
    image, mask = imageio.imread(image_path), imageio.imread(mask_path)
    fig, ax = plt.subplots(1, 2)
    ax[0].axis("off")
    ax[0].imshow(image, cmap="gray")
    # visualize the masks with random colors
    ax[1].axis("off")
    ax[1].imshow(mask, cmap=get_random_colors(mask), interpolation="nearest")
    plt.show()

In [ ]:
# plot the first 4 images
for i in range(4):
    plot_sample(image_paths[i], mask_paths[i])

You should see 4 images with nuclei and the corresponding colored nuclei masks. To further understand and explain the data, please answer the following QUESTIONS:
- Do the ids assigned to the individual nuclei (= colors in the 2nd column of images) have a fixed meaning? In other words, would the segmentation change if we change shuffle these numbers, so that all pixels labeled `1` are labeled `2` and so on?
- Can we directly use a U-Net to predict the mask ids? Why / Why not?
- Find the unique shapes of the images in the training set.
- Do all images have the same shape? If not, can this lead to problems? How could we deal with potentially arising problems?

In [8]:
# TODO find how many images we have in the training set
# and find all (unique) shapes of the images in the training set
shape = []
for i in range(len(mask_paths)):
  shape.append(imageio.imread(mask_paths[i]).shape)
unique = []
for x in shape:
    if x not in unique:
        unique.append(x)
print(unique)

[(256, 256), (360, 360), (260, 347), (520, 696), (512, 640), (256, 320)]


**Answer**

- The colors do not have a fixed meaning. The numbers are just similar per nuclei and zero stands for non-nuclei
- No, as size of the input images differ.
- Unique shapes: [(256, 256), (360, 360), (260, 347), (520, 696), (512, 640), (256, 320)]
- Yes as images have to have same size for U-Net. This can be solved by subsampling larger images to size of smalles images.

In [9]:
# TODO now load the images and masks into memory, and normalize the images so that they have zero mean and unit variance
images = [np.array(imageio.imread(path)) for path in image_paths]
masks = [np.array(imageio.imread(path).astype(dtype = 'uint8')) for path in mask_paths]

In [10]:
ims_flat = np.concatenate([im.ravel() for im in images])
mean, std = np.mean(ims_flat), np.std(ims_flat)
images = [(im.astype("float32") - mean) / std for im in images]

In [11]:
# TODO make a train validation split, use every 20th image for validation
train_images = [x for i, x in enumerate(images) if i%20 != 0]
train_masks = [x for i, x in enumerate(masks) if i%20 != 0]
val_images = [x for i, x in enumerate(images) if i%20 == 0]
val_masks = [x for i, x in enumerate(masks) if i%20 == 0]

In [36]:
# finally, let's choose the appropriate torch device
device = torch.device("cuda")
# device = torch.device("cpu")

**Implement PyTorch dataset**

As a next step, we implement a `torch.utils.data.Dataset` to have access to our data during training. As you shoulf have realized in the previous question, the images in our training set come in different sizes. In order to concatenate batches, we however need images of the same size. To solve this issue, we will subsample patches of size 256 x 256 (smallest image size) from the images in the dataset.

In [13]:
class NucleiDataset(Dataset):
    def __init__(self, images, masks, image_transform=None, mask_transform=None, transform=None):
        assert len(images) == len(masks)
        self.images = images
        self.masks = masks
        self.image_transform = image_transform
        self.mask_transform = mask_transform
        self.transform = transform

    def __getitem__(self, index):
        image, mask = self.images[index], self.masks[index]

        # crop the images to have the shape 256 x 256, so that we can feed them into memory
        # despite them having different sizes
        crop_shape = (256, 256)
        shape = image.shape
        if shape != crop_shape:
            assert image.ndim == mask.ndim == 2
            crop_start = [np.random.randint(0, sh - csh) if sh != csh else 0 for sh, csh in zip(shape, crop_shape)]
            crop = tuple(slice(cs, cs + csh) for cs, csh in zip(crop_start, crop_shape))
            image, mask = image[crop], mask[crop]
              
        # apply the transforms if given
        if self.image_transform is not None:
            image = self.image_transform(image)
        if self.mask_transform is not None:
            mask = self.mask_transform(mask)
        if self.transform is not None:
            image, mask = self.transform(image, mask)
        
        # make sure we have numpy arrays and add a channel dimension for the image data
        image, mask = np.array(image), np.array(mask)
        if image.ndim == 2:
            image = image[None]
        return image, mask
        
    def __len__(self):
        return len(self.images)

As discussed in the lecture, we can't learn the instance segmentation directly. So we will first start with a semantic segmentation problem and learn foreground background segmentation with the U-Net, i.e. predict for each pixel whether it belongs to a nucleus or to the background. To this end, implement a `mask_transform` that converts the instance segmentation ground-truth into a binary target and pass it to the training dataset.

In [14]:
def mask_transform(mask):
  m_b = (mask != 0) * 1
  mask_binary = np.expand_dims(m_b, axis = 0)
  return mask_binary

In [15]:
# TODO implement a transform that outputs the binary target 
# and instantiate the training dataset and validation dataset with it
# HINT: the transform can be a function that takes the mask array as input and returns the binarized version
# HINT: you will also need to add a channel dimension to the target
train_dataset = NucleiDataset(train_images, train_masks, image_transform=None, mask_transform=mask_transform, transform=None)
val_dataset = NucleiDataset(val_images, val_masks, image_transform=None, mask_transform=mask_transform, transform=None)

In [ ]:
# TODO sample a few images from the dataset and verify that they are correct
for i in range(9):
  fig, ax = plt.subplots(1, 2)
  ax[0].axis("off")
  ax[0].imshow(torch.tensor(train_dataset[i][0]).permute(2,1,0).squeeze(axis=2))
  ax[1].axis("off")
  ax[1].imshow(torch.tensor(train_dataset[i][1]).permute(2,1,0).squeeze(axis=2))
  plt.show()

The images you sample should look similar to this (using the standard colormap for the binarized mask):


### Implement the U-Net

Next, we will implement a U-Net architecture inspired [Ronneberger et al.](https://arxiv.org/abs/1505.04597), see the image below.

IMPORTANT: unlike in the publication / image we will use SAME convolutions, so that the input shape is the same as the output shape. This will introduce some boundary artifacts, but it will make implementing the rest of this exercise much simpler.
It also means that you don't need to crop the features passed on in the skip connections and can directly concatenate them.


HINTS: 
- the `up-conv` used here is called [ConvTranspose2d](https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html#convtranspose2d) in pytorch.
- you can concatenate the features coming from the skip connectins and the encoder with `torch.cat`
- you will need `nn.ModuleList` to store the individual blocks of the encoder / decoder

Before you implement the U-Net, please answer the following QUESTIONS:
- What kind of architecture is the U-Net and what is the rationale behind this architecture?
- What other types of architectures for semantic segmentation exist to achieve similar effects?
- Why are the skip connections used in the U-Net?

TODO your answers here
- Encoder-decoder architecture, learn semantic seg. in encoder, upsample semantic segmentation in decoder
- Multiscale image pyramid methods, Deep atrous convolutions, DeepLab combines all these
- To preserve high res information from the encoder layers that would otherwise be lost in the decoder

In [17]:
# TODO implement the U-Net architecture
class Block(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding = 'same')
        self.relu  = nn.ReLU()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding = 'same')
    
    def forward(self, x):
        return self.relu(self.conv2(self.relu(self.conv1(x))))

In [18]:
enc_block = Block(1, 64)
enc_block(torch.tensor(np.expand_dims(train_dataset[0][0], axis = 0))).shape

torch.Size([1, 64, 256, 256])

In [19]:
class Encoder(nn.Module):
    def __init__(self, chs=(1,64,128,256,512,1024)):
        super().__init__()
        self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)])
        self.pool       = nn.MaxPool2d(2)
    
    def forward(self, x):
        ftrs = []
        for block in self.enc_blocks:
            x = block(x)
            ftrs.append(x)
            x = self.pool(x)
        return ftrs

In [21]:
encoder = Encoder()
# input image
x    = torch.tensor(train_dataset[0][0])
ftrs = encoder(x)
print(x.shape)
for ftr in ftrs: print(ftr.shape)

torch.Size([1, 256, 256])
torch.Size([64, 256, 256])
torch.Size([128, 128, 128])
torch.Size([256, 64, 64])
torch.Size([512, 32, 32])
torch.Size([1024, 16, 16])


In [22]:
train_dataset[0][0].shape

(1, 256, 256)

In [23]:
ftrs[-2].shape

torch.Size([512, 32, 32])

In [24]:
class Decoder(nn.Module):
    def __init__(self, chs=(1024, 512, 256, 128, 64)):
        super().__init__()
        self.chs        = chs
        self.upconvs    = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs)-1)])
        self.dec_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs)-1)]) 
        
    def forward(self, x, encoder_features):
        for i in range(len(self.chs)-1):
            x        = self.upconvs[i](x)
            x        = torch.cat([x, encoder_features[i]], dim=1)
            x        = self.dec_blocks[i](x)
        return x

In [25]:
# TODO implement the U-Net architecture
class UNet(nn.Module):
    def __init__(self, enc_chs=(1,64,128,256,512,1024), dec_chs=(1024, 512, 256, 128, 64), num_class=1):
        super().__init__()
        self.encoder     = Encoder(enc_chs)
        self.decoder     = Decoder(dec_chs)
        self.head        = nn.Conv2d(dec_chs[-1], num_class, 1)

    def forward(self, x):
        enc_ftrs = self.encoder(x)
        out      = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out      = self.head(out)
        return out

In [37]:
# TODO instantiate a uner and check that your architecture is correct by applying it to
# an input from the train loader
model = UNet()
model.to(device)

UNet(
  (encoder): Encoder(
    (enc_blocks): ModuleList(
      (0): Block(
        (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (relu): ReLU()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      )
      (1): Block(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (relu): ReLU()
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
      )
      (2): Block(
        (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (relu): ReLU()
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
      )
      (3): Block(
        (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (relu): ReLU()
        (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=same)
      )
      (4): Block(
        (conv1): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), 

In [27]:
torch.tensor(train_dataset[0][1]).permute(2,1,0).squeeze(axis=2).squeeze(axis=0)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [ ]:
# TODO display the prediction. Can you interpret the resulting image?
pred = model(torch.tensor(np.expand_dims(train_dataset[0][0], axis = 0))).permute(1,2,3,0).squeeze(axis=3).squeeze(axis = 0)
plt.imshow(pred.detach().numpy())

## Training

Now we can define the training functions, similar to the last exercise. Unlike in the last exercises we will now use [tensorboard](https://www.tensorflow.org/tensorboard) to monitor the loss, metrics and images during training and validation. We can open it directly in colab (see the cell below). Note that the data for tensorboard is stored separately from the notebook file, so if you copy the notebook somewhere else you will not see the curves in tensorboard anymore. For this exercise the tensorboard files are not part of the submission and we only include it to help you monitor the training 

In [29]:
# in this cell all the training and validation functions are implemented for you already;
# these implementations are fairly similar to the exercise where we trained a CNN on CIFAR,
# but they now use tensorboard to log loss, metric and also images

# normalize a tensor to range [0, 1] (per channel).
# this is needed to properly display the tensor as an image in tensorboard
def normalize(tensor):
  eps = 1e-6
  normed = tensor.numpy()
  minval = normed.min(axis=(0, 2, 3), keepdims=True)
  normed = normed - minval
  maxval = normed.max(axis=(0, 2, 3), keepdims=True)
  normed = normed / (maxval + eps)
  return torch.from_numpy(normed)


# add a tensor as image to the tensorboard
def add_image(logger, tag, tensor, step):
  im = tensor.detach().cpu()
  if im.ndim == 3:
      im = normalize(im[None])
      logger.add_images(tag=tag, img_tensor=im, global_step=step)
  elif im.ndim == 4 and im.shape[1] in (1, 3):
      im = normalize(im)
      logger.add_images(tag=tag, img_tensor=im, global_step=step)
  elif im.ndim == 4:
      im = normalize(im)
      for c in range(im.shape[1]):
          logger.add_images(tag=f"{tag}-channel-{c}", img_tensor=im[:, c:c+1], global_step=step)
  else:
      raise ValueError(f"Expected 3 or 4d input tensor, got {im.ndim}")
      

# run the whole training
def run_training(
    model, train_loader, val_loader, loss, metric, optimizer, n_epochs, logger
):
    epoch_len = len(train_loader)
    step = 0
    for epoch in tqdm.trange(n_epochs):
        train_epoch(model, train_loader, loss, metric, optimizer, logger, step)
        step = epoch_len * (epoch + 1)
        validate(model, val_loader, loss, metric, logger, step)


# train the model for one epoch
def train_epoch(model, loader, loss, metric, optimizer, logger, step_begin):
    model.train()
    for i, (x, y) in enumerate(loader):
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss_value = loss(pred, y.float())
        loss_value.backward()
        optimizer.step()
        if metric is not None:
            metric_value = metric(pred, y)
        
        # log to the tensorboard
        step = step_begin + i
        # add the current loss and metric to the tensorboard
        logger.add_scalar(tag="train-loss", scalar_value=loss_value.item(), global_step=step)
        if metric is not None:
            logger.add_scalar(tag="train-metric", scalar_value=metric_value.item(), global_step=step)
        if step % 50 == 0:  # log images every 50 steps (every step would be too expensive)
            add_image(logger, "input", x, step)
            add_image(logger, "target", y, step)
            add_image(logger, "prediction", pred, step)


# validate the model
def validate(model, loader, loss, metric, logger, step):
    model.eval()
    n_val = len(loader)
    metric_value, loss_value = 0.0, 0.0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss_value += loss(pred, y.float()).item()
            if metric is not None:
                metric_value += metric(pred, y).item()
        
    metric_value /= n_val
    loss_value /= n_val
    # log to the tensorboard
    logger.add_scalar(tag="val-loss", scalar_value=loss_value, global_step=step)
    if metric is not None:
        logger.add_scalar(tag="val-metric", scalar_value=metric_value, global_step=step)

In [ ]:
# start the tensorboard in your notebook
%tensorboard --logdir runs

We will use (per-pixel) binary cross entropy as a loss function. Pytorch offers to different implementations of this loss: [BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html) and [BCEWithLogitsLoss](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html).

QUESTIONS:
- What is the difference between the two different implementations?
- Which one do we need to use here, and why?

TODO your answers here
- one takes values after activation (sigmoid), the other before activation
- we use BCEWithLogits, because the U-Net does not have a activation (preferable for numeric reasons)

In [31]:
# TODO create the loss function
loss = nn.BCEWithLogitsLoss()

Next, we want to define a suitable metric for measuring the quality of the (binary) network prediction. We will use the [dice score](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient) for this, which compares the intersection of prediction and target to their union.

In [54]:
# TODO implement the dice score as a function.
# HINTS: 
# - for later parts of this exercises, you should implement it in such a way that
# the function can compute the dice score for input and target with multiple channels,
# and so that it is computed independently per channel and the channel average is returned
# - since we don't have an activation in the U-Net you need to bring the predictions in range [0, 1] using torch.sigmoid
# - the dice score can be formulated for continuous predictions in [0, 1]; DO NOT threshold the predictions

def single_dice_coef(input, target):
    # shape of y_true and y_pred_bin: (height, width)
    input = torch.sigmoid(input)
    c = torch.sum(target * input) / torch.sum(target * torch.sigmoid(input))
    if torch.sum(target * torch.sigmoid(input)) == 0:
      c = 1
    intersection = torch.sum(target * input)
    if (torch.sum(target)==0) and (torch.sum(input)==0):
        return 1
    return (2*intersection) / (c * (torch.sum(target) + torch.sum(input)))

def dice_score(input, target):
    channel_num = target.shape[1]
    mean_dice_channel = 0
    for j in range(channel_num):
      channel_dice = single_dice_coef(input[0, j, :, :], target[0, j, :, :])
      mean_dice_channel += channel_dice/(channel_num)
    return mean_dice_channel

In [ ]:
# TODO check your dice implementation for an example model prediction and the corresponding target
for i in range(20):
  print(dice_score(model(torch.tensor(np.expand_dims(train_dataset[i][0], axis = 0))), torch.tensor(np.expand_dims(train_dataset[i][1], axis = 0))))

In [34]:
# this instantiates tensorboard logger, the logs for this model will be stored in
# 'runs/unet-1' and the model will be called 'unet-1' in tensorbaord
logger = SummaryWriter("runs/unet-1")

In [38]:
# train the model for 10 epochs, 
# during and after the training, check the tensorboard to see how the model loss and metrics evolve,
# and to see predictions of the training data in real time
n_epochs = 10
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True) 
val_loader = DataLoader(val_dataset, batch_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
run_training(model, train_loader, val_loader, loss, dice_score, optimizer, n_epochs, logger)

100%|██████████| 10/10 [05:56<00:00, 35.61s/it]


## Evaluate on test data

Now, we will evaluate the **instance segmentation**  derived from the model predictions on the test data. To this end, we will use [connected components](https://en.wikipedia.org/wiki/Component_(graph_theory)#Algorithms) to convert the (binarized) network predictions into an instance segmentation. We will use the implementation from [skimage](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.label).

QUESTION:
- Describe briefly what the connected components algorithm does when applied to a (binary) image. (You don't need to comment on the 'how' (implementation)).

TODO: your answer here
assigns unique instances to all pixels of non-touching foreground objects

In [40]:
# TODO load and normalize the test images
image_paths = glob(os.path.join(data_folder, "test", "images", "*.tif"))
image_paths.sort()
mask_paths = glob(os.path.join(data_folder, "test", "masks", "*.tif"))
mask_paths.sort()
assert len(image_paths) == len(mask_paths)
images = [np.asarray(imageio.imread(path)) for path in image_paths]
masks = [np.asarray(imageio.imread(path)) for path in mask_paths]
ims_flat = np.concatenate([im.ravel() for im in images])
images = [(im.astype("float32") - mean) / std for im in images]
test_images = images
test_masks = masks

In [ ]:
# check out instance segmentation for a few test images
counter = 0
with torch.no_grad():
    for im, mask in zip(test_images, test_masks):
        if counter > 3:
            break
        # predict with the model and apply sigmoid to map the prediction to the range [0, 1]
        pred = model(torch.from_numpy(im[None, None]).to(device))
        pred = torch.sigmoid(pred).cpu().numpy().squeeze()
        # get tbe nucleus instance segmentation by applying connected components to the binarized prediction
        nuclei = label(pred > 0.5)
        fig, ax = plt.subplots(1, 4, figsize=(16, 16))
        ax[0].axis("off")
        ax[0].imshow(im, cmap="gray")
        ax[1].axis("off")
        ax[1].imshow(mask, cmap=get_random_colors(mask), interpolation="nearest")
        ax[2].axis("off")
        ax[2].imshow(pred, cmap="gray")
        ax[3].axis("off")
        ax[3].imshow(nuclei, cmap=get_random_colors(nuclei), interpolation="nearest")
        plt.show()
        counter += 1

In [42]:
# we also need a measure for the instance segmentation quality.
# here, we use intersection over union (for each ground-truth object).
# it is implemented for you already below

def precision(tp, fp, fn):
    return tp / (tp + fp) if tp > 0 else 0


def compute_ious(seg, mask):
    overlap = contingency_table(seg, mask).toarray()
    n_pixels_pred = np.sum(overlap, axis=0, keepdims=True)
    n_pixels_true = np.sum(overlap, axis=1, keepdims=True)
    eps = 1e-7
    ious = overlap / np.maximum(n_pixels_pred + n_pixels_true - overlap, eps)
    # ignore matches with zero (= background)
    ious = ious[1:, 1:]
    n_pred, n_true = ious.shape
    n_matched = min(n_pred, n_true)
    return n_true, n_matched, n_pred, ious

    
def compute_tps(ious, n_matched, threshold):
    not_trivial = n_matched > 0 and np.any(ious >= threshold)
    if not_trivial:
        # compute optimal matching with iou scores as tie-breaker
        costs = -(ious >= threshold).astype(float) - ious / (2*n_matched)
        pred_ind, true_ind = linear_sum_assignment(costs)
        assert n_matched == len(true_ind) == len(pred_ind)
        match_ok = ious[pred_ind, true_ind] >= threshold
        tp = np.count_nonzero(match_ok)
    else:
        tp = 0
    return tp


def intersection_over_union(seg, mask, threshold=0.5):
    if seg.sum() == 0:
        return 0.0
    n_true, n_matched, n_pred, ious = compute_ious(seg, mask)
    tp = compute_tps(ious, n_matched, threshold)
    fp = n_pred - tp
    fn = n_true - tp
    ap = precision(tp, fp, fn)
    return ap

In [43]:
# Compute the average IOU of all the test images
ious = []
model.eval()
with torch.no_grad():
    for im, mask in tqdm.tqdm(zip(test_images, test_masks), total=len(test_images)):
        # the model can only predict images with a spatial size that is divisible by 16
        # if it isn't we just cat a few pixels to make it fit
        if any(sh % 16 != 0 for sh in im.shape):
            crop = tuple(
                slice(0, -(sh%16)) for sh in im.shape
            )
            im = im[crop]
            mask = mask[crop]
        input_ = torch.from_numpy(im[None, None]).to(device)
        pred = model(input_)
        pred = torch.sigmoid(pred).cpu().numpy().squeeze()
        labels = label(pred > 0.5)
        iou = intersection_over_union(labels, mask)
        ious.append(iou)
print()
print("Mean IoU:", np.mean(ious))

100%|██████████| 50/50 [00:02<00:00, 18.10it/s]


Mean IoU: 0.5373021580515761


QUESTION:
- Given the samples of the instance segmentations from above, what are the biggest sources of error you can identify in the model predictions / instance segmentation approach?
- How could they be overcome?

TODO your answer here
- nuclei (objects) are touching and cannot be separated via connected components
- e.g. learn boundaries in addition and use them to separate the nuclei

## Train with boundary channel

To avoid merges of touching nuclei, we will now add a boundary channel to the learning objective, and then use it for object separation in the instance segmentation funtion. To this end, we will train a U-Net that outputs 3 channels (background, foreground, object boundary) and use the cross entropy loss.

In [44]:
# in this cell all the training and validation functions are implemented for you already;
# these implementations are fairly similar to the exercise where we trained a CNN on CIFAR,
# but they now use tensorboard to log loss, metric and also images

# normalize a tensor to range [0, 1] (per channel).
# this is needed to properly display the tensor as an image in tensorboard
def normalize(tensor):
  eps = 1e-6
  normed = tensor.numpy()
  minval = normed.min(axis=(0, 2, 3), keepdims=True)
  normed = normed - minval
  maxval = normed.max(axis=(0, 2, 3), keepdims=True)
  normed = normed / (maxval + eps)
  return torch.from_numpy(normed)


# add a tensor as image to the tensorboard
def add_image(logger, tag, tensor, step):
  im = tensor.detach().cpu()
  if im.ndim == 3:
      im = normalize(im[None])
      logger.add_images(tag=tag, img_tensor=im, global_step=step)
  elif im.ndim == 4 and im.shape[1] in (1, 3):
      im = normalize(im)
      logger.add_images(tag=tag, img_tensor=im, global_step=step)
  elif im.ndim == 4:
      im = normalize(im)
      for c in range(im.shape[1]):
          logger.add_images(tag=f"{tag}-channel-{c}", img_tensor=im[:, c:c+1], global_step=step)
  else:
      raise ValueError(f"Expected 3 or 4d input tensor, got {im.ndim}")
      

# run the whole training
def run_training(
    model, train_loader, val_loader, loss, metric, optimizer, n_epochs, logger
):
    epoch_len = len(train_loader)
    step = 0
    for epoch in tqdm.trange(n_epochs):
        train_epoch(model, train_loader, loss, metric, optimizer, logger, step)
        step = epoch_len * (epoch + 1)
        validate(model, val_loader, loss, metric, logger, step)


# train the model for one epoch
def train_epoch(model, loader, loss, metric, optimizer, logger, step_begin):
    model.train()
    for i, (x, y) in enumerate(loader):
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss_value = loss(pred, y)
        loss_value.backward()
        optimizer.step()
        if metric is not None:
            metric_value = metric(pred, y)
        
        # log to the tensorboard
        step = step_begin + i
        # add the current loss and metric to the tensorboard
        logger.add_scalar(tag="train-loss", scalar_value=loss_value.item(), global_step=step)
        if metric is not None:
            logger.add_scalar(tag="train-metric", scalar_value=metric_value.item(), global_step=step)
        if step % 50 == 0:  # log images every 50 steps (every step would be too expensive)
            add_image(logger, "input", x, step)
            add_image(logger, "target", y, step)
            add_image(logger, "prediction", pred, step)


# validate the model
def validate(model, loader, loss, metric, logger, step):
    model.eval()
    n_val = len(loader)
    metric_value, loss_value = 0.0, 0.0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss_value += loss(pred, y).item()
            if metric is not None:
                metric_value += metric(pred, y).item()
        
    metric_value /= n_val
    loss_value /= n_val
    # log to the tensorboard
    logger.add_scalar(tag="val-loss", scalar_value=loss_value, global_step=step)
    if metric is not None:
        logger.add_scalar(tag="val-metric", scalar_value=metric_value, global_step=step)

In [45]:
# TODO implement a transform that outputs an image with 3 values:
# 0 for backgorund, 1 for foreground (= nucleus) and 2 for boundary (boundary pixel between nucleus and background or between 2 nuclei)
# HINT: you can use skimage.segmentation.find_boundaries (already imported) to determine the boundary pixels
def label_transform(mask):
  mask = mask_transform(mask)
  mask_b = find_boundaries(mask) * 2
  mask = mask + mask_b
  mask[mask>2] = 2
  return mask.squeeze(axis = 0)

In [46]:
# instantiate the training and validation datasets with the new label transform
train_dataset = NucleiDataset(
    train_images, train_masks, mask_transform=label_transform
)
val_dataset = NucleiDataset(
    val_images, val_masks, mask_transform=label_transform
)

In [ ]:
# visualize the new label transform and make sure it's correct
counter = 0
for im, target in train_dataset:
    if counter > 3:
        break
    fig, ax = plt.subplots(1, 2, figsize=(16, 16))
    ax[0].axis("off")
    ax[0].imshow(im[0], cmap="gray")
    ax[1].axis("off")
    ax[1].imshow(target)
    plt.show()
    counter += 1

In [48]:
# TODO instantiate the new unet and loss function
model = UNet(enc_chs=(1,64,128,256,512,1024), dec_chs=(1024, 512, 256, 128, 64), num_class=3)
model.to(device)
loss = nn.CrossEntropyLoss()

In [49]:
# train the new U-Net for 10 epochs
# (we don't use a metric here, since the target (with class labels 0, 1, 2) and prediction (one-hot encoding) have different representations
n_epochs = 10
logger = SummaryWriter("runs/unet-2")
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True) 
val_loader = DataLoader(val_dataset, batch_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
metric = None
run_training(model, train_loader, val_loader, loss, metric, optimizer, n_epochs, logger)

100%|██████████| 10/10 [06:10<00:00, 37.04s/it]


In [ ]:
# TODO write instance segmentation function where you first subtract the  boundary from the foreground prediction,
# and then apply connected components to the result in order to get the instances
# the parameter 'threshold' should determine at which threshold the result is binarized before applying connected components 
# HINT: you can use skimage.segmentation.watershed (already implemented) afterwards to grow the segmentation back so that it
# covers the full prediction while keeping the instance labels
def instance_segmentation(foreground_prediction, boundary_prediction, threshold=0.5):

In [ ]:
# TODO check the prediction results and instance segmentation for a few images
# make sure your instance segmentation implementation is correct
# HINT: you need to apply a softmax to the network predictions 

In [ ]:
# TODO use the validation set to find a good value for the 'threshold' parameter in the instance_segmentation function
best_threshold = 

In [ ]:
# Compute the average IOU of all the test images
ious = []
model.eval()
with torch.no_grad():
    for im, mask in tqdm.tqdm(zip(test_images, test_masks), total=len(test_images)):
        # 
        if any(sh % 16 != 0 for sh in im.shape):
            crop = tuple(
                slice(0, -(sh%16)) for sh in im.shape
            )
            im = im[crop]
            mask = mask[crop]
        
        input_ = torch.from_numpy(im[None, None]).to(device)
        pred = model(input_)
        pred = torch.softmax(pred, dim=1).cpu().numpy().squeeze()
        assert pred.shape[0] == 3
        nuclei = instance_segmentation(pred[1], pred[2])

        iou = intersection_over_union(nuclei, mask, best_threshold)
        ious.append(iou)
print()
print(np.mean(ious))

QUESTIONS:
- Did training with the boundaries and using them in the instance segmentation improve the results?
- Can you think of ways to improve the loss function for this learning task?

TODO your answers here
- Yes, should have improved it, but depends a bit on the learning process...
- Boundary target is quite unbalanced, use a balanced loss function, e.g. Dice Loss

## [Optional] Try variations of the UNet architecture

- add BatchNorm
- replace TransposedConvs with bilinear upsampling and 1x1 convs
- use residual blocks instead of the normal convolutional blocks

In [ ]:
# TODO implement and compare u-net architecture variations

## [Optional] Train with data augmentation

- rotations & flips
- noise augmentations (add gaussian noise and/or poisson noise)
- crop and resize

Hint: use the `image_transform` (noise augmentations) and `transform` (geometrical augmentations) argument of the `NucleiDataset`

In [ ]:
# TODO implement data augmentations and compare with training without augmentations

# Denoising with Noise2Noise

Here, we implement [Noise2Noise](https://arxiv.org/abs/1803.04189), which is a method to learn image denoising **without** ground-truth, by using a different noisy image as target. Note that we are re-using the same architecture as before, which is a bit different from the one used in the paper.

## Data Preparation

We use the [VSDR dataset](https://cv.snu.ac.kr/research/VDSR), which contains 'clean' (i.e. without noise) natural images.

In [88]:
# Download the VSDR data
def download(url, output_folder):
    if os.path.exists(output_folder):
        return
    os.makedirs(output_folder, exist_ok=True)
    tmp_path = os.path.join(output_folder, "data.zip")
    with requests.get(url) as r:
        with open(tmp_path, "wb") as f:
            f.write(r.content)
    with zipfile.ZipFile(tmp_path, "r") as f:
        f.extractall(output_folder)
    os.remove(tmp_path)

train_url = "https://cv.snu.ac.kr/research/VDSR/train_data.zip"
test_url = "https://cv.snu.ac.kr/research/VDSR/test_data.zip"

vsdr_train = os.path.join(root_folder, "vsdr_train")
download(train_url, vsdr_train)
vsdr_test = os.path.join(root_folder, "vsdr_test")
download(test_url, vsdr_test)

In [87]:
del vsdr_train

In [56]:
# check the unique image sizes in the training data
image_exts = (".jpeg", ".jpg", ".png", ".bmp")
all_image_paths = [
    p for p in Path(vsdr_test).glob("**/*") if p.suffix.lower() in image_exts
]
shapes = []
for p in all_image_paths:
    shapes.append(imageio.imread(p).shape)
unique_shapes = list(set(shapes))
print(unique_shapes)

[(769, 1024, 3), (344, 228, 3), (576, 1024, 3), (321, 481, 3), (665, 1024, 3), (480, 500, 3), (362, 500, 3), (689, 1024, 3), (644, 1024, 3), (1024, 685, 3), (391, 586, 3), (276, 276, 3), (819, 1024, 3), (672, 1024, 3), (737, 1024, 3), (692, 1024, 3), (1024, 742, 3), (694, 1024, 3), (1024, 683, 3), (1200, 798, 3), (576, 720, 3), (1024, 700, 3), (1037, 778, 3), (1024, 768, 3), (805, 1024, 3), (616, 1024, 3), (681, 1024, 3), (512, 768, 3), (768, 1024, 3), (512, 512, 3), (288, 352, 3), (256, 256, 3), (512, 512), (683, 1024, 3), (1024, 1024, 3), (575, 1024, 3), (766, 1024, 3), (481, 321, 3), (963, 1280, 3), (660, 1024, 3), (835, 1024, 3), (727, 1024, 3), (656, 529, 3), (1024, 1005, 3), (773, 1024, 3), (827, 1200, 3), (980, 1000, 3), (732, 1024, 3), (797, 1024, 3), (864, 1024, 3), (673, 1024, 3), (1024, 692, 3), (695, 1024, 3), (1024, 567, 3), (280, 280, 3), (361, 250, 3), (656, 1024, 3), (676, 1024, 3), (680, 1024, 3), (678, 1024, 3), (682, 1024, 3), (765, 1024, 3), (288, 288, 3)]


In [57]:
# vsdr dataset that can deal with all the different data sizes in the training and test data
class VsdrDataset(Dataset):
    def __init__(self, root_dir, noise_transform):
        image_exts = (".jpeg", ".jpg", ".png", ".bmp")
        self.image_paths = [
            p for p in Path(root_dir).glob("**/*") if p.suffix.lower() in image_exts
        ]
        self.noise_transform = noise_transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        im = np.array(imageio.imread(self.image_paths[index]))
        if im.ndim == 2:
            im = np.concatenate([im[..., None]] * 3, axis=2)
        if im.shape[-1] == 1:
            im = np.concatenate([im] * 3, axis=2)
        target_shape = (256, 256)
        # first, pad the images if they are smaller than the target crop shape
        if any(sh < tsh for sh, tsh in zip(im.shape[:-1], target_shape)):
            padding = tuple((0, tsh - sh if sh < tsh else 0) for sh, tsh in zip(im.shape[:-1], target_shape))
            padding = padding + ((0, 0),)
            im = np.pad(im, padding, mode="reflect")
        # then crop tham if they are too large
        if any(sh > tsh for sh, tsh in zip(im.shape[:-1], target_shape)):
            crop_start = [np.random.randint(0, sh - tsh) if sh != tsh else 0 for sh, tsh in zip(im.shape[:-1], target_shape)]
            crop = tuple(slice(cs, cs + tsh) for cs, tsh in zip(crop_start, target_shape))
            im = im[crop]
        assert im.shape == (256, 256, 3), f"{im.shape}"
        
        # normalize the image to range [0, 1] (per channel)
        im = im.astype("float32")
        im -= im.min(axis=(0, 1), keepdims=True)
        im /= im.max(axis=(0, 1), keepdims=True)
        # bring image in range [-1, 1]
        im = 2 * im - 1
        
        # transform to channel first order
        im = im.transpose((2, 0, 1))
        # apply the noise transformation
        return self.noise_transform(im)

In [58]:
# go from range [-1, 1] to [0, 255] and uint 8
# and go to channel last order
def to_display(im):
    im = (((im + 1) / 2) * 255).astype("uint8")
    return im.transpose((1, 2, 0))

In [ ]:
# sample a few images
check_dataset = VsdrDataset(vsdr_test, lambda x: x)
fig, ax = plt.subplots(4, 4, figsize=(16, 16))
for ii in range(16):
    im = check_dataset[ii]
    ax[ii % 4, ii // 4].axis("off")
    ax[ii % 4, ii // 4].imshow(to_display(im))
plt.show()

## Train a network with clean targets

As the first baseline, we will train a network with clean target data (i.e. without noise added to it). 

In [77]:
# instatiate the model and loss function (Mean Squared Error)
model = UNet(enc_chs=(3,64,128,256,512,1024), dec_chs=(1024, 512, 256, 128, 64), num_class=1)
model.to(device)
loss = nn.MSELoss()
loss.to(device)

MSELoss()

In [78]:
# we will use the PSNR (peak signal to noise ratio) as metric to evaluate the predictions.
# you can read up on it and the implementation we are using here:
# https://www.ni.com/en-us/innovations/white-papers/11/peak-signal-to-noise-ratio-as-an-image-quality-metric.html
# (the higher the PSNR, the better)

def to_uint8(im):
    return torch.clamp((im + 0.5) * 255.0 + 0.5, 0, 255).type(torch.uint8)


class PSNR(nn.Module):
    def __call__(self, x, y):
        x, y = to_uint8(x), to_uint8(y)
        x, y = x.detach().cpu().numpy(), y.detach().cpu().numpy()
        return peak_signal_noise_ratio(x, y)

QUESTION:
- What does the PSNR metric measuere?
- Why is it used instead of 'just' computing the pixel wise error (MSE)?

TODO your answer here

In [79]:
# TODO implement a function that adds additive gaussian noise to the input
# the var argument should correspond to the variance of the gaussian
def additive_gaussian_noise(x, var):
  return x + np.random.normal(0,np.sqrt(var),x.shape)

In [ ]:
# TODO instantiate a vsdr dataset corresponding to the vsdr_train folder 
# with a noise transformation that returns a corrupted image (as network input)
# and a clean image (as target)
# use a variance of 0.1
# split the dataset into a train (90% of the data) and validationd dataset (10%)
# HINT: you can use torch.utils.data.random_split to split a torch dataset

vsdr_train_ds =
vsdr_val_ds =

In [ ]:
# sample a few images from the train dataset
fig, ax = plt.subplots(3, 2, figsize=(16, 16))
for ii in range(3):
    noisy, clean = vsdr_train_ds[ii]
    # go back to uint8
    ax[ii, 0].axis("off")
    ax[ii, 0].imshow(to_display(noisy))
    ax[ii, 1].axis("off")
    ax[ii, 1].imshow(to_display(clean))
plt.show()

In [ ]:
# train the denoising network
n_epochs = 10
logger = SummaryWriter("runs/noise2noise-1")
train_loader = DataLoader(vsdr_train_ds, batch_size=1, shuffle=True) 
val_loader = DataLoader(vsdr_val_ds, batch_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
metric = PSNR()
run_training(model, train_loader, val_loader, loss, metric, optimizer, n_epochs, logger)

In [ ]:
# create the test dataset
vsdr_test_ds = VsdrDataset(
    vsdr_test,
    noise_transform=lambda x: (additive_gaussian_noise(x, var=0.1), x)
)

In [ ]:
# TODO display some results on the test data

## Train a network with noisy targets (Gaussian noise)

Now, we train a network from noisy inputs AND noisy targets. And compare its results to the previous denoising network learned on clean targets.

QUESTION:
- Why can the model learn denoising even though we also have a noisy target?

TODO your answer here
- uncorrelated noise with zero mean does not affect the MSE loss (in limit to infinity)

In [ ]:
# instantiate new model and loss
model = UNet(in_channels=3, out_channels=3)
model.to(device)
loss = nn.MSELoss()
loss.to(device)

In [ ]:
# TODO create dataset with corrupted input and target
# use same variance as before (0.1)
vsdr_train_ds =
vsdr_val_ds =

In [ ]:
# train the model
n_epochs = 10
logger = SummaryWriter("runs/noise2noise-2")
train_loader = DataLoader(vsdr_train_ds, batch_size=1, shuffle=True) 
val_loader = DataLoader(vsdr_val_ds, batch_size=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
metric = PSNR()
run_training(model, train_loader, val_loader, loss, metric, optimizer, n_epochs, logger)

In [ ]:
# TODO show a few predictions for the test dataset

In [ ]:
# TODO compute the PSNR for the whole test dataset

QUESTIONS:
- Compare the result between the network trained with clean targets and with noisy targets.

TODO your answer here

## [Optional] Train a network with noisy targets (Poisson noise)

Use poisson noise instead of gaussian noise and compare to both the network trained with clean target and the network with the target corrupted by gaussian noise.

In [ ]:
# TODO run training with poisson noise and compare with previous results

## [Optional] Use U-Net with BatchNorm

Repeat the exercise with a U-Net with BatchNorm. Does this improve the results?

In [ ]:
# TODO use a U-Net architecture with BatchNorm and compare with previous results